In [25]:
import pandas as pd
import requests
import streamlit.components.v1 as components
import folium
from sodapy import Socrata
import datetime as dt
import plotly.express as px


In [26]:
client = Socrata("opendata.rdw.nl", 'fWzun7gN4kA2grz6Tef9NPtIN', username='max.verbruggen@hva.nl', password='KealoEcko718')

results = client.get("8ys7-d773", query='SELECT kenteken, brandstof_omschrijving LIMIT 14392104')
rdw1 = pd.DataFrame.from_records(results)



KeyboardInterrupt: 

In [ ]:
results = client.get('m9d7-ebf2', query='SELECT kenteken, datum_eerste_toelating LIMIT 15088755')
rdw2 = pd.DataFrame.from_records(results)

In [ ]:
rdw1

,kenteken,brandstof_omschrijving
0,0001TJ,Benzine
1,0001TV,Benzine
2,0001VH,Benzine
3,0001VM,Benzine
4,0002GV,Diesel
...,...,...
14392099,ZV743D,Benzine
14392100,ZV743F,Benzine
14392101,ZV743G,Benzine
14392102,ZV743H,Benzine


In [ ]:
rdw2

,kenteken,datum_eerste_toelating
0,J096VZ,20201102
1,J096XB,20201228
2,J096XF,20201103
3,J096XG,20201029
4,J096XJ,20201120
...,...,...
15088750,J078GT,20200227
15088751,J078GV,20200218
15088752,J078GX,20201223
15088753,J078GZ,20200731


In [ ]:
rdw = rdw1.merge(rdw2, on='kenteken')

In [ ]:
rdw = rdw[rdw['datum_eerste_toelating'].notna()]
rdw.isna().sum()

kenteken                  0
brandstof_omschrijving    0
datum_eerste_toelating    0
dtype: int64

In [37]:
rdw_test = rdw.copy()

In [ ]:
for fuel in rdw_test.brandstof_omschrijving.unique():
    globals()[fuel.lower()] = rdw_test[rdw_test['brandstof_omschrijving'] == fuel].reset_index(drop=True)

data = pd.DataFrame()
data['datum_eerste_toelating'] = rdw_test['datum_eerste_toelating'].unique()
data['benzine_count'] = 0
data['elektriciteit_count'] = 0
data['diesel_count'] = 0
data['waterstof_count'] = 0
data['alcohol_count'] = 0
data['lpg_count'] = 0
data['cng_count'] = 0
data['lng_count'] = 0

data = data.set_index('datum_eerste_toelating')

In [69]:
for i, row in data.iterrows():
    data.loc[i]['benzine_count'] = len(benzine[benzine['datum_eerste_toelating'] == i].index)
    data.loc[i]['elektriciteit_count'] = len(elektriciteit[elektriciteit['datum_eerste_toelating'] == i].index)
    data.loc[i]['diesel_count'] = len(diesel[diesel['datum_eerste_toelating'] == i].index)
    data.loc[i]['waterstof_count'] = len(waterstof[waterstof['datum_eerste_toelating'] == i].index)
    data.loc[i]['alcohol_count'] = len(alcohol[alcohol['datum_eerste_toelating'] == i].index)
    data.loc[i]['lpg_count'] = len(lpg[lpg['datum_eerste_toelating'] == i].index)
    data.loc[i]['cng_count'] = len(cng[cng['datum_eerste_toelating'] == i].index)
    data.loc[i]['lng_count'] = len(lng[lng['datum_eerste_toelating'] == i].index)

for column in data.columns:
    data[column] = data[column].cumsum()

data

,benzine_count,elektriciteit_count,diesel_count,waterstof_count,alcohol_count,lpg_count,cng_count,lng_count
datum_eerste_toelating,,,,,,,,
1877-06,0,1,0,0,0,0,0,0
1889-06,0,1,2,0,0,0,0,0
1894-06,1,1,3,0,0,0,0,0
1895-06,2,1,3,0,0,0,0,0
1897-02,3,1,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...
2022-11,10259556,1154458,2597855,842,4250,147341,14243,1274
2022-12,10279374,1178377,2603780,845,4250,147619,14247,1280
2023-01,10310277,1197400,2612075,850,4250,147919,14250,1285


In [70]:
data.to_csv('fuel_count.csv')